# Modelling - Regression

This notebook explores regression models to **predict Length of Stay (days)** as a baseline to the [Long Stayer Risk Stratification](https://github.com/nhsx/skunkworks-long-stayer-risk-stratification) model which achieved a Mean Absolute Error **(MAE) of 3.8 days** (2.2 median absolute error).

This notebook is broken down into:

1. Statistical tests to check the validity of linear models using Ordinary Least Squares (OLS)
2. Training a range of baseline models using cross validation
3. Testing final models on a test dataset
4. Exploring in more detail the best performing baseline model

In [ ]:
import pandas as pd
import pickle
import numpy as np
import math
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.model_selection import train_test_split, GridSearchCV
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn import linear_model
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from scipy.stats import kstest, shapiro, anderson
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

%matplotlib inline
plt.rcParams["figure.figsize"] = [15, 8]

In [ ]:
# Helper functions


def train_model(gsc, X_train, y_train):
    """Uses a GridSearchCV instance to find a reasonable model, and store
    performance and fitted model into a python dict

    Parameters:

        gsc (sklearn.model_selection.GridSearchCV object): defined model
        X_train (pandas dataframe): training dataframe with features
        y_train (pandas dataframe): training dataframe with targets

    Returns:

        (dict): resulting fitted model and performance metrics
    """

    grid_result = gsc.fit(X_train, y_train)

    # note model fitted/scored on MSE
    model = {
        "cv_mse_mean": np.round(
            -grid_result.cv_results_["mean_test_score"][grid_result.best_index_], 3
        ),
        "cv_mse_std": np.round(
            grid_result.cv_results_["std_test_score"][grid_result.best_index_], 2
        ),
        "model": grid_result.best_estimator_,
    }

    # retrain the best estimator on the full training set - note that refit=True does not appear to do this
    # note we calculate MAE as final metric
    model["model"].fit(X_train, y_train)
    model["mae"] = np.round(
        mean_absolute_error(y_train, model["model"].predict(X_train)), 3
    )

    return model


def risk_score(los):
    """Return risk score (1-5) based on LoS

    Parameters:
        los (float): length of stay in days

    Returns:
        (int): risk score (1 = Very low risk, 5 = High risk)
    """

    # round los up to whole days
    los = math.ceil(los)

    if los > 15:
        return 5
    elif los > 13:
        return 4
    elif los > 10:
        return 3
    elif los > 6:
        return 2
    else:
        return 1

## Load features

In [ ]:
features = pd.read_parquet("../../data/features.parquet")
features.shape

## Cap length of stay

The highest length of stay is ~250 days, so we will check the distribution of length of stay and cap high values.

In [ ]:
# Check distribution of length of stay
features.groupby(by="LENGTH_OF_STAY").count().AGE_ON_ADMISSION.plot();

In [ ]:
# Cap maximum length of stay to 30 days
features.LENGTH_OF_STAY = features.LENGTH_OF_STAY.apply(lambda x: 30 if x > 30 else x)

## Define target and training features

In [ ]:
X = features.drop(columns="LENGTH_OF_STAY")
y = features.LENGTH_OF_STAY

## Variance Inflaction Factors

Simple linear models (ordinary least squares) assume there is no multi-collinearity.

Variance inflaction factors (VIF) help quantify the extent of any collinearity present.

We are looking for VIF ~< 10 across our features.

In [ ]:
# Takes ~6 minutes to run on a STANDARD_DS3_V2
vif = pd.DataFrame()
vif["feature"] = X.columns

vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]
vif

## Residual distributions of OLS

If VIF factors indicate a lack of multi-collinearity (they do not), check for normality of residuals aka homoescadisticity.

This requires training a linear model, calculating the residuals and checking visually and statistically that they are normally distributed.

In [ ]:
# Train basic OLS model for statistical testing only
reg = linear_model.LinearRegression()
reg.fit(X, y)
pred = pd.Series(reg.predict(X))
resid = y - pred

In [ ]:
# Visual inspection
resid.hist(bins=30);

### Shapiro-Wilk test for normality

* Null hypothesis = our residuals are drawn from normal distribution
* Alternate hypothesis = our residuals are not drawn from normal distribution (and fail requirements of OLS model)
* Test statistic shows how much distribution differs to normal distribution
* p-value is probability null hypothesis true
* p-value < 0.05 leads us to reject null hypothesis

In [ ]:
shapiro(resid)

### One-sided Kolmogorov-Smirnov test for normality

* Null hypothesis = our residuals are drawn from normal distribution
* Alternate hypothesis = our residuals are not drawn from normal distribution (and fail requirements of OLS model)
* Test statistic shows how much distribution differs to normal distribution
* p-value is probability null hypothesis true
* p-value < 0.05 leads us to reject null hypothesis

In [ ]:
kstest(resid, "norm")

### Anderson-Darling test for normality

* Null hypothesis = our residuals are drawn from normal distribution
* Alternate hypothesis = our residuals are not drawn from normal distribution (and fail requirements of OLS model)
* Test statistic is compared to critical value at the significance level required (e.g. 5%)
* Test statistic > critical value for 5% significance level leads us to reject null hypothesis

In [ ]:
anderson(resid, "norm")

**Statistical testing invalidate assumptions for OLS models**

## Train/test split

For model evaluation, we will hold back a 25% test set, and use cross-validation on the remaining 75% for all models until the final comparison is made.

In [ ]:
# Split data for train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.25, random_state=42
)

# Modeling

Strategy is to try a number of regression models with:

* GridsearchCV for hyperparameter tuning with cross validation, refitting full training set for best model
* Test all final models against the held-out test set.

OLS models are excluded due to statistical assumptions not being met. NN are excluded due to complexity/interpretability issues.

In [ ]:
# Initiate empty models dictionary
models = {}

## Mean model

The simplest baseline model takes the mean length of stay as its prediction

In [ ]:
model_name = "mean"

# define gridsearch parameters
gsc = GridSearchCV(
    estimator=DummyRegressor(strategy="mean"),
    param_grid={},
    cv=5,
    scoring="neg_mean_squared_error",
    verbose=1,
    n_jobs=-1,
    refit=True,
)

# takes ~1 seconds to run on a STANDARD_DS3_V2
models[model_name] = train_model(gsc, X_train, y_train)
models[model_name]

### Elastic net regression

A regularised linear model.

In [ ]:
model_name = "elastic"

# define gridsearch parameters
gsc = GridSearchCV(
    estimator=linear_model.ElasticNet(),
    param_grid={
        "l1_ratio": [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1],
    },
    cv=5,
    scoring="neg_mean_squared_error",
    verbose=1,
    n_jobs=-1,
    refit=True,
)

# takes ~2 seconds to run on a STANDARD_DS3_V2
models[model_name] = train_model(gsc, X_train, y_train)
models[model_name]

### Random forest regressor

In [ ]:
model_name = "randomforest"

# define gridsearch parameters
gsc = GridSearchCV(
    estimator=RandomForestRegressor(),
    param_grid={"n_estimators": [10, 100, 1000], "max_depth": [5, 10, None]},
    cv=5,
    scoring="neg_mean_squared_error",
    verbose=1,
    n_jobs=-1,
    refit=True,
)

# takes ~15 mins to run on a STANDARD_DS3_V2
models[model_name] = train_model(gsc, X_train, y_train)
models[model_name]

### Catboost

Boosted tree optimised for categorical features

In [ ]:
model_name = "catboost"

# extract categorical features
num_features = [
    "AGE_ON_ADMISSION",
    "EL CountLast12m",
    "EMCountLast12m",
    "OP First CountLast12m",
    "OP FU CountLast12m",
]
cat_features = list(set(X_train.columns) - set(num_features))

# define gridsearch parameters
gsc = GridSearchCV(
    estimator=CatBoostRegressor(verbose=False, cat_features=cat_features),
    param_grid={
        "max_depth": [5, 10, None],
        "learning_rate": [0.01, 0.1, 1],
        "iterations": [10, 100, 1000],
    },
    cv=5,
    scoring="neg_mean_squared_error",
    verbose=1,
    n_jobs=-1,
    refit=True,
)

# takes ~7 mins to run on a STANDARD_DS3_V2
models[model_name] = train_model(gsc, X_train, y_train)
models[model_name]

### XGBoost

In [ ]:
model_name = "xgboost"

# define gridsearch parameters
gsc = GridSearchCV(
    estimator=XGBRegressor(random_state=42),
    param_grid={
        "n_estimators": [1, 5],
        "learning_rate": [0.01, 0.1, 1],
        "max_depth": [5, 10, None],
    },
    cv=5,
    scoring="neg_mean_squared_error",
    verbose=1,
    n_jobs=-1,
    refit=True,
)

# takes ~1 mins to run on a STANDARD_DS3_V2
models[model_name] = train_model(gsc, X_train, y_train)
models[model_name]

## Save models

In [ ]:
# save models outside the git tree
with open("../../models/regression.pickle", "wb") as handle:
    pickle.dump(models, handle)

## Load models

In [ ]:
# load models from outside the git tree
with open("../../models/regression.pickle", "rb") as handle:
    models = pickle.load(handle)
models

## Validate models

Use the held-out test set to evaluate the performance of all the tuned models

In [ ]:
for model in models:
    # ensure smallest LoS is 0 days (not negative value)
    preds = np.clip(models[model]["model"].predict(X_test), 0, None)
    # calculate MAE and range of LoS
    mae = mean_absolute_error(y_test, preds)
    print(
        f"{model} test mae: {mae.round(3)} ({preds.min().round(1)} - {preds.max().round(1)} days)"
    )

## Model exploration

A single performance metric can be a misleading summary of how a model performs. We will take the "best performing" baseline model, and explore in more detail how the model performs.

Todo:

- [ ] Retrain simpler model with only top ~10 features?
- [ ] Fairness analysis

In [ ]:
model = "catboost"
actual = y_test.reset_index(drop=True)  # extract actual LoS
preds = pd.Series(
    np.clip(models[model]["model"].predict(X_test), 0, None)
)  # cast to series for consistency
mae = mean_absolute_error(actual, preds)
print(f"MAE: {mae}, min LoS: {preds.min()}, max LoS: {preds.max()}")

### Actual vs predicted plot

The maximum length of stay the model can predict is ~9 days, which is well short of the 30 day actual. 

Let's visualise model performance:

In [ ]:
# plot predicted vs actual
plt.scatter(x=preds, y=actual, alpha=0.1)
# plot ideal 1:1 prediction line. Max LoS = 30
plt.plot(np.arange(0, 31), np.arange(0, 31), "r--")
plt.xlabel("Predicted LoS")
plt.ylabel("Actual LoS")
# scale to ~[0,30] for consistency with other plots
plt.xlim([-1, 31])
plt.ylim([-1, 31]);

### Relative error by length of stay plot

In [ ]:
# Create dataframe with actual values
errors_df = pd.DataFrame(data=actual).reset_index(drop=True)
# Calculate relative error
errors_df["error"] = errors_df.LENGTH_OF_STAY - preds
# Plot relative error
errors_df.plot.scatter(x="LENGTH_OF_STAY", y="error", alpha=0.1)
# Plot mean relative error and 95% confidence intervals
# Flag: errors are not normally distibuted so does 2*std capture 5%?
plt.plot(np.arange(0, 31), np.ones(31) * errors_df.error.mean(), "r")
plt.plot(
    np.arange(0, 51),
    np.ones(51) * (errors_df.error.mean() + (2 * errors_df.error.std())),
    "g--",
)
plt.plot(
    np.arange(0, 51),
    np.ones(51) * (errors_df.error.mean() - (2 * errors_df.error.std())),
    "g--",
)
plt.xlim([-1, 31])
# scale plot
plt.legend(
    [
        f"\u03bc ({np.round(errors_df.error.mean(),2)} days)",
        f"95% LoA (\u03C3 {np.round(errors_df.error.std(),2)} days gives {2*np.round(errors_df.error.std(),2)})",
    ]
);

### Risk score equivalence

In addition to a length of stay prediction (days), we will define a risk score of a patient becoming a long stayer:

Risk Category|Day Range for Risk Category
-----|------
1 - Very low risk|0-6
2 - Low risk|7-10
3 - Normal risk|11-13
4 - Elevated risk|14-15
5 - High risk|>15

We can convert the length of stay prediction into a risk score, and visualise the performance of this approach. Later, we will look at classification models to do this directly.

#### Calculate risk scores

In [ ]:
# calculate risk scores in a new dataframe
risk = pd.DataFrame()
# extract LoS values
risk["los"] = actual
risk["los_predicted"] = preds
# calculate equivalent risk categories
risk["risk"] = risk.los.apply(lambda x: risk_score(x))  # actual risk
risk["risk_predicted"] = risk.los_predicted.apply(
    lambda x: risk_score(x)
)  # risk as predicted by regression

#### Calculate errors in risk stratification

In [ ]:
risk_labels = [
    "1 - Very low risk",
    "2 - Low risk",
    "3 - Normal risk",
    "4 - Elevated risk",
    "5 - High risk",
]
risks = dict.fromkeys(risk_labels)
for r in risks:
    risks[r] = np.array([0.0, 0.0, 0.0, 0.0, 0.0])  # create empty values

    for label in risk_labels:
        this_risk = int(label[0])
        # extract the real los for each predicted risk category
        subset = risk[risk.risk_predicted == this_risk]

        if r == "1 - Very low risk":
            prop = (subset.los < 7).sum() / subset.shape[0]
        elif r == "2 - Low risk":
            prop = (subset[subset.los >= 7].los <= 10).sum() / subset.shape[0]
        elif r == "3 - Normal risk":
            prop = (subset[subset.los >= 11].los <= 13).sum() / subset.shape[0]
        elif r == "4 - Elevated risk":
            prop = (subset[subset.los >= 14].los <= 15).sum() / subset.shape[0]
        elif r == "5 - High risk":
            prop = (subset.los > 15).sum() / subset.shape[0]

        risks[r][this_risk - 1] = prop

#### Plot actual risk vs predicted

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
bottom = np.array([0.0, 0.0, 0.0, 0.0, 0.0])
for r in risks:
    if r == "1 - Very low risk":
        data = risks[r]
        ax.bar(risk_labels, data, label=r, width=0.35)
    else:
        bottom += data
        data = risks[r]
        ax.bar(risk_labels, data, label=r, bottom=bottom, width=0.35)
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], risk_labels[::-1], bbox_to_anchor=(1.05, 1))
ax.set_ylabel("Actual risk category")
ax.set_xlabel("Predicted risk cateogry");